In [ ]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows

in_path = "C:\\Users\\OneDrive\\Desktop\\PCLCONUS\\Input\\PCL\\"
input_filename = 'pcl_west_wgs.tif' 


THEFOLDER = "C:\\Users\\\OneDrive\\Desktop\\GlobalPCL"
out_path = f"{THEFOLDER}\\PCLTILES\\"
output_filename = 'pcltile_{}-{}.tif'

widthtile = 5000
heighttile = 5000

def get_tiles(ds, width=widthtile, height=heighttile):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

os.makedirs(out_path, exist_ok=True)

tile_numbers = []

with rio.open(os.path.join(in_path, input_filename)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata
    meta = inds.meta.copy()
    for window, transform in get_tiles(inds):
        data = inds.read(window=window)
        if nodata is not None and not (data == nodata).all():
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            tile_number = f"{int(window.col_off)}-{int(window.row_off)}"
            tile_numbers.append(tile_number)
            outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(data)

# Print or store the tile numbers
TILENUMBER = tile_numbers

del in_path, input_filename, tile_numbers
del out_path, output_filename, widthtile, heighttile, tile_width, tile_height
del meta, nodata, window, inds, get_tiles